In [3]:
# =========================================================
# GOOGLE COLAB — Script para calcular a tabela kw_resumo
# (Kruskal–Wallis + epsilon-squared) a partir de um Excel (.xlsx)
#
# Como usar no Colab:
# 1) Rode esta célula.
# 2) Faça upload do arquivo dados.xlsx quando solicitado.
# 3) O script imprime:
#    - DataFrame kw_resumo
#    - Bloco LaTeX pronto para colar no paper
# =========================================================

!pip -q install openpyxl scipy pandas numpy

from google.colab import files
import io
import pandas as pd
import numpy as np
from scipy.stats import kruskal

# ---------- Upload ----------
uploaded = files.upload()  # selecione seu dados.xlsx
if not uploaded:
    raise RuntimeError("Nenhum arquivo foi enviado. Faça upload do seu .xlsx.")

# pega o primeiro arquivo enviado
fname = next(iter(uploaded.keys()))
print("Arquivo carregado:", fname)

df = pd.read_excel(io.BytesIO(uploaded[fname]))
df.columns = [c.strip() for c in df.columns]
print("Colunas encontradas:", list(df.columns))

# ---------- Config (ajuste se seu arquivo usar outros nomes) ----------
GROUP_COL_CANDIDATES = ["Grupo", "GRUPO", "group", "Group"]

OUTCOME_COLS_CANDIDATES = [
    "Requisitos de Arquitetura",
    "Desenvolvimento do Sistema",
    "Desenvolvimento Sistema",
    "MEDIA TDE",
    "MÉDIA TDE",
    "PROVA OBJETIVA E AUTORIA",
    "NOTA FINAL GERAL ( TDE 40% + PROVA 60%)",
    "NOTA FINAL GERAL",
]

# ---------- Helpers ----------
def find_first_existing_col(df: pd.DataFrame, candidates: list[str]) -> str:
    for c in candidates:
        if c in df.columns:
            return c
    raise KeyError(
        f"Nenhuma das colunas candidatas foi encontrada.\n"
        f"Candidatas: {candidates}\n"
        f"Colunas no arquivo: {list(df.columns)}"
    )

def to_numeric_series(s: pd.Series) -> pd.Series:
    # converte strings com vírgula/espacos; força NaN quando não numérico
    s2 = (
        s.astype(str)
         .str.replace("\u00a0", " ", regex=False)  # NBSP
         .str.replace(",", ".", regex=False)
         .str.strip()
    )
    return pd.to_numeric(s2, errors="coerce")

def epsilon_squared_from_H(H: float, k: int, N: int) -> float:
    """
    Epsilon-squared para Kruskal–Wallis:
      eps2 = (H - k + 1) / (N - k)
    Pode ficar negativo quando H < k-1; truncamos para 0.
    """
    if N <= k:
        return float("nan")
    eps2 = (H - k + 1) / (N - k)
    return float(max(0.0, eps2))

def escape_latex(s: str) -> str:
    repl = {
        "\\": r"\textbackslash{}",
        "&": r"\&",
        "%": r"\%",
        "$": r"\$",
        "#": r"\#",
        "_": r"\_",
        "{": r"\{",
        "}": r"\}",
        "~": r"\textasciitilde{}",
        "^": r"\textasciicircum{}",
    }
    out = str(s)
    for k, v in repl.items():
        out = out.replace(k, v)
    return out

def fmt_p(p: float) -> str:
    if pd.isna(p):
        return "--"
    if p < 0.001:
        return "<0.001"
    return f"{p:.3f}"

def fmt_eps(eps: float) -> str:
    if pd.isna(eps):
        return "--"
    return f"{eps:.3f}"

# ---------- Detect columns ----------
group_col = find_first_existing_col(df, GROUP_COL_CANDIDATES)

outcome_cols = [c for c in OUTCOME_COLS_CANDIDATES if c in df.columns]
if not outcome_cols:
    raise KeyError(
        f"Nenhuma coluna de desfecho encontrada.\n"
        f"Candidatas: {OUTCOME_COLS_CANDIDATES}\n"
        f"Colunas no arquivo: {list(df.columns)}"
    )

print("Coluna de grupo selecionada:", group_col)
print("Colunas de desfecho encontradas:", outcome_cols)

# ---------- Compute KW per outcome ----------
df[group_col] = df[group_col].astype(str).str.strip()

rows = []
for outcome in outcome_cols:
    tmp = df[[group_col, outcome]].copy()
    tmp[outcome] = to_numeric_series(tmp[outcome])

    tmp = tmp.dropna(subset=[outcome])
    tmp = tmp[tmp[group_col].notna() & (tmp[group_col] != "")]

    groups = []
    group_names = []
    for g, sub in tmp.groupby(group_col):
        vals = sub[outcome].to_numpy(dtype=float)
        if len(vals) > 0:
            groups.append(vals)
            group_names.append(g)

    k = len(groups)
    N = int(tmp.shape[0])

    if k < 2:
        H = np.nan
        p = np.nan
        eps2 = np.nan
    else:
        H, p = kruskal(*groups, nan_policy="omit")
        eps2 = epsilon_squared_from_H(H, k, N)

    rows.append(
        {
            "Variável": outcome,
            "Grupos (k)": k,
            "N": N,
            "H (KW)": float(H) if pd.notna(H) else np.nan,
            "p (KW)": float(p) if pd.notna(p) else np.nan,
            "epsilon^2": float(eps2) if pd.notna(eps2) else np.nan,
            "Grupos encontrados": ", ".join(group_names),
        }
    )

kw_df = pd.DataFrame(rows)

print("\n=== kw_resumo (Kruskal–Wallis) ===")
display(kw_df[["Variável", "Grupos (k)", "N", "p (KW)", "epsilon^2", "Grupos encontrados"]])

# ---------- Generate LaTeX table ----------
latex_lines = []
latex_lines.append(r"\begin{table}[t]")
latex_lines.append(r"\centering")
latex_lines.append(r"\small")
latex_lines.append(r"\caption{Resultados inferenciais (Kruskal--Wallis) por variável de desempenho, comparando grupos de personalidade (16Personalities). Reporta-se $p$-value e tamanho de efeito $\epsilon^2$ (epsilon-squared).}")
latex_lines.append(r"\label{tab:kw_resumo}")
latex_lines.append(r"\begin{tabular}{lrrrr}")
latex_lines.append(r"\toprule")
latex_lines.append(r"\textbf{Variável} & \textbf{Grupos (k)} & \textbf{N} & \textbf{$p$ (KW)} & \textbf{$\epsilon^2$} \\")
latex_lines.append(r"\midrule")

for _, r in kw_df.iterrows():
    var = escape_latex(r["Variável"])
    k = int(r["Grupos (k)"]) if pd.notna(r["Grupos (k)"]) else 0
    N = int(r["N"]) if pd.notna(r["N"]) else 0
    p = fmt_p(r["p (KW)"])
    eps = fmt_eps(r["epsilon^2"])
    latex_lines.append(f"{var} & {k} & {N} & {p} & {eps} \\\\")

latex_lines.append(r"\bottomrule")
latex_lines.append(r"\end{tabular}")
latex_lines.append(r"\end{table}")

latex_table = "\n".join(latex_lines)

print("\n=== LaTeX (cole no paper) ===\n")
print(latex_table)

# (Opcional) salvar a tabela em arquivo .tex para download
out_tex = "kw_resumo.tex"
with open(out_tex, "w", encoding="utf-8") as f:
    f.write(latex_table)

print(f"\nArquivo LaTeX salvo: {out_tex}")
files.download(out_tex)


Saving dados.xlsx to dados (1).xlsx
Arquivo carregado: dados (1).xlsx
Colunas encontradas: ['Turma', 'MBTI', 'ANO', 'GENDER', 'TDE1 (Requistos Arquiettura)', 'TDE2 (Requistos Arquiettura)', 'Média Requisitos de Arquitetura', 'TDE3 (Desenvolvimento Sistema)', 'Desenvolvimento Sistema', 'PROVA OBJETIVA E AUTORIA', 'MEDIA TDE', 'TDE 40%', 'PROVA 60%', 'NOTA FINAL GERAL ( TDE 40% + PROVA 60%)', 'Grupo', 'Perfil']
Coluna de grupo selecionada: Grupo
Colunas de desfecho encontradas: ['Desenvolvimento Sistema', 'MEDIA TDE', 'PROVA OBJETIVA E AUTORIA', 'NOTA FINAL GERAL ( TDE 40% + PROVA 60%)']

=== kw_resumo (Kruskal–Wallis) ===


,Variável,Grupos (k),N,p (KW),epsilon^2,Grupos encontrados
0,Desenvolvimento Sistema,4,102,0.769247,0.0,"Analistas, Diplomatas, Exploradores, Sentinelas"
1,MEDIA TDE,4,102,0.746663,0.0,"Analistas, Diplomatas, Exploradores, Sentinelas"
2,PROVA OBJETIVA E AUTORIA,4,102,0.786363,0.0,"Analistas, Diplomatas, Exploradores, Sentinelas"
3,NOTA FINAL GERAL ( TDE 40% + PROVA 60%),4,102,0.822087,0.0,"Analistas, Diplomatas, Exploradores, Sentinelas"



=== LaTeX (cole no paper) ===

\begin{table}[t]
\centering
\small
\caption{Resultados inferenciais (Kruskal--Wallis) por variável de desempenho, comparando grupos de personalidade (16Personalities). Reporta-se $p$-value e tamanho de efeito $\epsilon^2$ (epsilon-squared).}
\label{tab:kw_resumo}
\begin{tabular}{lrrrr}
\toprule
\textbf{Variável} & \textbf{Grupos (k)} & \textbf{N} & \textbf{$p$ (KW)} & \textbf{$\epsilon^2$} \\
\midrule
Desenvolvimento Sistema & 4 & 102 & 0.769 & 0.000 \\
MEDIA TDE & 4 & 102 & 0.747 & 0.000 \\
PROVA OBJETIVA E AUTORIA & 4 & 102 & 0.786 & 0.000 \\
NOTA FINAL GERAL ( TDE 40\% + PROVA 60\%) & 4 & 102 & 0.822 & 0.000 \\
\bottomrule
\end{tabular}
\end{table}

Arquivo LaTeX salvo: kw_resumo.tex


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>